In [25]:
import csv
import psycopg2

from psycopg2 import sql
import xml.etree.ElementTree as ET

In [20]:
conn = psycopg2.connect(
    host="localhost",
    database="gpx",
    user="root",
    password="root"
)

In [18]:
def kml2csv(kml_path, csv_path):
    with open(csv_path, mode='w') as csv_file:
        csv_writer = csv.writer(csv_file)

        tree = ET.parse(kml_path)
        root = tree.getroot()

        ns = {"kml": "http://www.opengis.net/kml/2.2"}
        placemarks = root.findall(".//kml:Placemark", ns)

        for placemark in placemarks:
            name = placemark.find("kml:name", ns)
            if name is None:
                continue

            coords = placemark.find(".//kml:coordinates", ns)
            if coords is None:
                continue

            points = coords.text.strip().split()
            for point in points:
                coordinates = point.split(",")

                latitude = float(coordinates[1])
                longitude = float(coordinates[0])

                csv_writer.writerow([name.text, latitude, longitude])

In [15]:
kml2csv('../user_data/points.kml', '../data/points.csv')
kml2csv('../user_data/segments.kml', '../data/segments.csv')

In [56]:
def csv2postgis(table_name, csv_path):
    try:
        cursor = conn.cursor()

        create_table = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    latitude DECIMAL,
    longitude DECIMAL
);
"""
        cursor.execute(create_table)
        conn.commit()

        with open(csv_path, 'r', ) as csv_file:
            reader = csv.reader(csv_file)
            for row in reader:
                (name, lat, lon) = row
                
                insert_query = f"""
INSERT INTO {table_name} 
(name, latitude, longitude) 
VALUES ('{name}', {lat}, {lon});
"""
                cursor.execute(
                    insert_query
                )
                conn.commit()
                
    except Exception as e:
        print(f'Error: {e}')
        conn.rollback()

In [58]:
csv2postgis('points', '../data/points.csv')
csv2postgis('segments', '../data/segments.csv')